<a href="https://colab.research.google.com/github/SeanSDarcy2001/CISProgrammingAssignments/blob/main/CIS1HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Integrated Surgery: Programming Assignment 1

In [122]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import files
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import least_squares # is this a valid numerical library?

## PART 1
Develop (or develop proficiency with) a Cartesian math package for 3D points, rotations, and frame transformations.

In [11]:
## use package to represent 3D points
p1    = np.array([-1489., -4913.,  4345.])
p2    = np.array([ 2633., -3268.,  5249.])
pcalc = np.array([-3210., -4390.,  3930.])

def norm(v):
    return v / np.sqrt(np.dot(v, v))

In [2]:
## use packages to represent 3D rotations
def identity():
  return np.array([[1, 0,  0], [0, 1, 0], [0, 0, 1]])

# generate skew matrix from 1x3 array 
def skew(v) :
  skewArg = norm(v)
  ax = skewArg[0]
  ay = skewArg[1]
  az = skewArg[2]
  skew = np.array([[0,    -az,  ay],
                   [az,   0,    -ax],
                   [-ay,  ax,   0]])
  return skew

# generate 3D Rotation matrix
# @Params: alpha, beta, gamma - Rotation angles about the x, y, and z axes respectively
# @Returns: 3D Rotation matrix
def RotfromAngles(alpha, beta, gamma) :
  Rx = np.array([[1, 0,  0], [0, np.cos(alpha), -np.sin(alpha)], [0, np.sin(alpha), np.cos(alpha)]])
  Ry = np.array([[np.cos(beta), 0,  np.sin(beta)], [0, 1, 0], [-np.sin(beta), 0, np.cos(beta)]])
  Rz = np.array([[np.cos(gamma), -np.sin(gamma),  0], [np.sin(gamma), np.cos(gamma), 0], [0, 0, 1]])
  R = RotfromComponents(Rx, Ry, Rz)
  return R

def smallAngleR(v, theta) :
  n = norm(v)
  return (identity() + skew(theta * n))

def RotfromComponents(Rx, Ry, Rz) :
  R = np.dot(np.dot(Rx, Ry), Rz)
  return R

# convert 3D vector into 4D homogeneous coords
def homogenousVector(v, scale) :
  Vx = scale * v[0]
  Vy = scale * v[1]
  Vz = scale * v[2]
  return np.array([Vx, Vy, Vz, scale])

R = RotfromAngles(0, 1, 0)
print(R)

# theta = int
# RxTheta = np.array([1, 0,  0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]))
# RyTheta

[[ 0.54030231  0.          0.84147098]
 [ 0.          1.          0.        ]
 [-0.84147098  0.          0.54030231]]


In [3]:
## use packages to represent 3D frame transformations

class Frame:
  
  def __init__(self, R, p):
    self.R = R
    self.p = p
    self.F = self.getFrame()

  # define Frame using Rotation and Translation
  # @Params: Rotation R and Translation p
  # @Return: Frame
  def getFrame(self):
    F = np.row_stack((np.column_stack((self.R, self.p)),
                      [0,0,0,1]))
    return F

  # applies frame transformation to vector
  # @Params: Frame F and Vector v
  # @Return: transformed vector
  def appFrame(self, v):
    return np.dot(self.F, v)

  # applies inverse frame transformation to vector
  # @Params: Frame F and Vector v
  # @Return: transformed vector
  def appInvFrame(self, v):
    Finv = Frame(self.R.T, - (np.dot(self.R.T, self.p)))
    return np.dot(Finv.getFrame(), v)

# test cases
p = np.array([5, 7, 9])
F = Frame(R, p)
print(F.getFrame())

one_t = np.array([[0,0,0],[1,0,0],[0,1,0]])
two_t = np.array([[0,0,1], [1,0,1],[0,0,2]])
# Transformation is 90 degree rot over x and +1 translation on z
# Tested here
R = RotfromAngles(90, 0, 0)
p = (0,0,1)
F = Frame(R, p)
n = homogenousVector(one_t[0],1)
print((n))
print(F.appFrame(n))

[[ 0.54030231  0.          0.84147098  5.        ]
 [ 0.          1.          0.          7.        ]
 [-0.84147098  0.          0.54030231  9.        ]
 [ 0.          0.          0.          1.        ]]
[0 0 0 1]
[0. 0. 1. 1.]


In [4]:
# ## example?
# n1, n2, ncalc = [norm(p) for p in [p1, p2, pcalc]]

# new_zaxis  = norm(np.cross(n1, n2))
# new_xaxis  = n1
# zero       = np.zeros(3) 

# fig = plt.figure(figsize=[10, 8])
# ax  = fig.add_subplot(1, 1, 1, projection='3d')

# x, y, z = zip(zero, new_xaxis)
# plt.plot(x, y, z, '-k', linewidth=3)

# x, y, z = zip(zero, new_zaxis)
# plt.plot(x, y, z, '--k', linewidth=3)

# x, y, z = zip(zero, n2)
# plt.plot(x, y, z, '-r', linewidth=1)

# x, y, z = zip(zero, ncalc)
# plt.plot(x, y, z, '--g', linewidth=1)

# ax.set_xlim(-1, 1)
# ax.set_ylim(-1, 1)
# ax.set_zlim(-1, 1)

# plt.show()

## PART 2

Develop a 3D point set to 3D point set registration algorithm

K. Arun, et. al., IEEE PAMI, Vol 9, no 5, pp 698-700, Sept 1987 

In [5]:
def rigid_registration(a, b):
    # Calculate and subtract means
    a_c = np.expand_dims(np.mean(a, axis = 1), axis=1)
    b_c = np.expand_dims(np.mean(b, axis = 1), axis=1)
    A = a - a_c
    B = b - b_c

    # Compute SVD
    H = np.dot(A, B.T)
    U, X, V_t = np.linalg.svd(H)
    R = np.dot(V_t.T, U.T)
    assert np.allclose(np.linalg.det(R), 1)
    p = b_c - np.matmul(R, a_c)
    return Frame(R, p)

In [6]:
# Two sets of points to check 3D-3D registration
one_t = np.array([[0,0,0],[1,0,0],[0,1,0]])
two_t = np.array([[0,0,1], [1,0,1],[0,0,2]])

reg = rigid_registration(one_t.T, two_t.T)
res = np.dot(reg.R, one_t.T) + reg.p
if np.allclose(res, two_t.T):
    print("Success!")
else:
    print("Registration failed.")

Success!


## PART 3

Develop a “pivot” calibration method

Yaniv 2015 <https://dx.doi.org/10.1117/12.2081348>

In [10]:
def pivotCalibration(frames) :
    """
    :param frames: N x 4 x 4 ndarray.
    :returns: pointer offset, pivot point and RMS Error about centroid of pivot.
    """
    err = -1.0
    trans = frames[:, 0:3, 3]

    # find pivot point in world coordinates using sphere fitting
    pivot = np.concatenate([np.mean(trans, axis=0), np.zeros(1)])

    # Compute least squares optimization for unknown point
    x_s = trans[:, 0]
    y_s = trans[:, 1]
    z_s = trans[:, 2]
    res = least_squares(func, pivot,
                        bounds=((-np.inf, -np.inf, -np.inf, -np.inf),
                                (np.inf, np.inf, np.inf, np.inf)),
                        jac='3-point',
                        args=(x_s, y_s, z_s))

    ptr_p = res.x[0:3]

    # Calculate mean offset
    offsets = np.zeros((frames.shape[0], 3))

    rot = frames[:, 0:3, 0:3]
    for i, rot in enumerate(rot):
        offsets[i] = rot.transpose() @ (ptr_p - trans[i])
    ptr_o = np.mean(offsets, axis=0)

    # Calculate residual error (root mean square error)

    x_v = np.concatenate([ptr_o, ptr_p],axis=0).reshape((6, 1))
    num = frames.shape[0]

    # Concatenate rotation and -I for each frame
    a_0 = (frames[:, 0:3, 0:3]).reshape((3*num,3))
    a_1 = (np.eye(3) * -1.0).reshape((1, 3, 3)).repeat(num, 0).reshape((3*num,3))
    a_v = np.concatenate((a_0, a_1), axis=1)

    # -1 * translation for each frame
    b_v = (frames[:, 0:3, 3] * -1.0).reshape(((3*num,1)))

    # Compute residuals
    resid = (np.dot(a_v, x_v) - b_v)
    err = np.sqrt(np.mean(resid * resid))


    return ptr_o, ptr_p, err


# Function to calculate distance of given
# points from center and radius given
# by the pivot.
def func(pvt, x_s, y_s, z_s):
    # Calculate distance from given center
    dist = numpy.sqrt((x_s - pvt[0])**2 +
                      (y_s - pvt[1])**2 +
                      (z_s - pvt[2])**2)

    return dist - pvt[3]

## Part 4

Given a distortion calibration data set, as described above, compute the “expected” values $\overrightarrow{C_i}^{(expected)}$ for the $\overrightarrow{C_i}$:

a. For each calibration data frame $[\overrightarrow{D_1}, ..., \overrightarrow{D_{N_D}}, \overrightarrow{A_1}, ..., \overrightarrow{A_{N_A}}, \overrightarrow{C_1}, ..., \overrightarrow{C_{N_C}}]$, compute the transformation between optical tracker $F_D$ and EM tracker coordinates. I.e., compute a frame $F_D$ such that $\overrightarrow{D_j} = F_D \cdot \overrightarrow{d_j}$.

b. Similarly, compute a transformation $F_A$ between calibration object and optical tracker coordinates. I.e., $\overrightarrow{A_j} = F_A \cdot \overrightarrow{a_j}$.

c. Given $F_D$ and $F_A$, compute $\overrightarrow{C_i}^{(expected)} = F_D^{-1} \cdot F_A \cdot \overrightarrow{c_j}$.

d. Output $\overrightarrow{C_i}^{(expected)}$ (see file formats below)

In [118]:
def read_calbody(name):
    path = "https://raw.githubusercontent.com/SeanSDarcy2001/CISProgrammingAssignments/main/data/" + name + "-calbody.txt"
    dat = pd.read_csv(path)

    N_D, N_A, N_C, FILE = dat.columns
    N_D = int(N_D)
    N_A = int(N_A) + N_D
    N_C = int(N_C) + N_A

    dat = dat.to_numpy()
    d_coords = dat[0:N_D,   0:3]
    a_coords = dat[N_D:N_A, 0:3]
    c_coords = dat[N_A:N_C, 0:3]

    # Outputs marker types x marker points x coords
    return np.array([d_coords, a_coords, c_coords])


def read_calreadings(name):
    path = "https://raw.githubusercontent.com/SeanSDarcy2001/CISProgrammingAssignments/main/data/" + name + "-calreadings.txt"
    dat = pd.read_csv(path)

    N_D, N_A, N_C, N_F, FILE = dat.columns
    N_D = int(N_D)
    N_A = int(N_A) + N_D
    N_C = int(N_C) + N_A
    N_F = int(float(N_F))

    dat = dat.to_numpy()
    D_coords = np.array([dat[0:N_D,   0:3]])
    A_coords = np.array([dat[N_D:N_A, 0:3]])
    C_coords = np.array([dat[N_A:N_C, 0:3]])
    for n in range(1, N_F):
        D_coords = np.append(D_coords,
                  np.array([dat[((n*N_C)):((n*N_C) + N_D),       0:3]]),
                  axis=0)
        A_coords = np.append(A_coords,
                  np.array([dat[((n*N_C) + N_D):((n*N_C) + N_A), 0:3]]),
                  axis=0)
        C_coords = np.append(C_coords,
                  np.array([dat[((n*N_C) + N_A):((n*N_C) + N_C), 0:3]]),
                  axis=0)

    # Outputs marker types x frames x marker points x coords
    return np.array([D_coords, A_coords, C_coords])

In [119]:
def read_empivot(name):
    path = "https://raw.githubusercontent.com/SeanSDarcy2001/CISProgrammingAssignments/main/data/" + name + "-empivot.txt"
    dat = pd.read_csv(path)

    N_D, N_F, FILE = dat.columns
    N_D = int(N_D)
    N_F = int(float(N_F))

    dat = dat.to_numpy()
    D_coords = np.array([dat[0:N_D,   0:3]])
    for n in range(1, N_F):
        print(np.array([dat[((n*N_D)):((n*N_D) + N_D),       0:3]]).shape)
        D_coords = np.append(D_coords,
                  np.array([dat[((n*N_D)):((n*N_D) + N_D),       0:3]]),
                  axis=0)

    # Outputs marker types x frames x marker points x coords
    return np.array([D_coords])

def read_optpivot(name):
    path = "https://raw.githubusercontent.com/SeanSDarcy2001/CISProgrammingAssignments/main/data/" + name + "-optpivot.txt"
    dat = pd.read_csv(path)

    N_D, N_H, N_F, FILE = dat.columns
    N_D = int(N_D)
    N_H = int(N_H) + N_D
    N_F = int(float(N_F))

    dat = dat.to_numpy()
    D_coords = np.array([dat[0:N_D,   0:3]])
    A_coords = np.array([dat[N_D:N_H, 0:3]])
    for n in range(1, N_F):
        print(np.array([dat[((n*N_H)):((n*N_H) + N_D),       0:3]]).shape)
        D_coords = np.append(D_coords,
                  np.array([dat[((n*N_H)):((n*N_H) + N_D),       0:3]]),
                  axis=0)
        H_coords = np.append(H_coords,
                  np.array([dat[((n*N_H) + N_D):((n*N_H) + N_H), 0:3]]),
                  axis=0)

    # Outputs marker types x frames x marker points x coords
    return np.array([D_coords, H_coords])

In [123]:
def read_output(name):
    path = "https://raw.githubusercontent.com/SeanSDarcy2001/CISProgrammingAssignments/main/data/" + name + "-output1.txt"
    dat = pd.read_csv(path)

    N_C, N_F, FILE = dat.columns
    N_C = int(N_C)
    N_F = int(float(N_F))

    dat = dat.to_numpy()
    em_probe = np.array(dat[0, 0:3])
    opt_probe = np.array(dat[1, 0:3])
    C_coords = np.array([dat[2:N_C,   0:3]])
    for n in range(1, N_F):
        print(np.array([dat[((2+n*N_C)):((2+n*N_C) + N_C),       0:3]]).shape)
        C_coords = np.append(C_coords,
                  np.array([dat[((2+n*N_C)):((2+n*N_C) + N_C),       0:3]]),
                  axis=0)

    # Outputs list of em_probe, opt_probe, frames x marker points x coords
    return [em_probe, opt_probe, C_coords]

def write_output(name, em_pt, opt_pt, C_exp):
    path = name + '-output1.txt'
    with open(path, 'w') as f:
        f.write(int(C_exp.shape[1]) + ', ' +
                int(C_exp.shape[0]) + ', ' + path)
        cw = csv.writer(f, delimiter=',')
        cw.writerow(em_pt)
        cw.writerow(opt_pt)
        for frame in range(C_exp.shape[0]):
            for marker in range(C_exp.shape[1]):
                cw.writerow(C_exp[frame][marker])
    files.download(path)

## Part 5

## Part 6

# Contributions

## Seby


*   developed proficiency with a Cartesian math package for 3D points rotations, and frame transformations. (1)


## Alex


*   developed proficiency with a Cartesian math package for 3D points rotations, and frame transformations. (1)
*   created Frame object with transformation functionality. (1)
*   developed algorithm for 3D-3D registration. (2)
*   developed method for pivot calibration using sphere fitting. (3)
*   developed methods for reading in files for application. (4-6)



#callibration object

#N_c is set of EM tracked points on callibration object relative to calibration object frame Fc (known)
#N_a is set of optically tracked points on callibration object relative to calibration object frame Fc (known)
#N_d are optically tracked points on base of EM tracker relative to EM base (known)
#A_j are optically tracked points on callibration object relative to optical tracker (highly accurate)
#D_j are optically tracked points on base of EM tracker relative to optical tracker (highly accurate)
#C_i are measured positions of EM tracked points on callibration object relative to EM tracker, subject to distortion and noise v.

#data is [D, A, C]
#wish to recover 

#pointer probes

#N_h is set of optically tracked points on one pointer probe relative to pointer frame (fixed, unknown)
#N_g is set of EM tracked points on other pointer probe relative to pointer frame (fixed, unknown)
#H_i is set of optically tracked points on pointer probe relative to optical tracker (highly accurate)
#G_i is set of EM tracked points on other pointer probe relative to EM tracker, subject to distortion and noise v.

#data is [D, H] and [G]
#wish to recover 
#wish to recover position of pivot dimple relative to EM tracker base 